# DLthon - 길찾는 오리들
## Base Settings

In [1]:
# importing the libraries
import keras, tensorflow as tf
from keras import layers

keras.mixed_precision.set_global_policy('mixed_float16')

# importing the modules
import models_base, models_top, utils

2025-01-21 13:08:23.459600: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-01-21 13:08:23.479307: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1737432503.496728  261232 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1737432503.501546  261232 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-01-21 13:08:23.523302: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

In [2]:
# loading the dataset
train_dataset, val_dataset, test_dataset = utils.load_unified_datasets("data/data_disruptive_feature_exclusion")

Found 374 files belonging to 6 classes.
Using 225 files for training.
Using 149 files for validation.


I0000 00:00:1737432507.686934  261232 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 9571 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3080, pci bus id: 0000:01:00.0, compute capability: 8.6


In [3]:
# designing the model
def eV2L_simple2(): # this can be adjusted to be hyper-tunable
    inputs = keras.Input(shape=(224, 224, 3))
    x = utils.image_preprocessing_v1()(inputs) # image preprocessing
    x = models_base.EV2L((224, 224, 3))(x, training=False) # pre-trained model
    x = models_top.simple2()(x) # top model
    outputs = layers.Dense(6, activation='softmax', name="classifier")(x)

    model = keras.Model(inputs, outputs)
    
    model.compile(loss='categorical_crossentropy',
                   optimizer=keras.optimizers.RMSprop(learning_rate=1e-4),
                   metrics=['accuracy'])
    
    return model

eV2L_simple2().summary()

Model: "functional_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ image_preprocessing_v1          │ (None, 224, 224, 3)    │             7 │
│ (Sequential)                    │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ efficientnetv2-l (Functional)   │ (None, 7, 7, 1280)     │   117,746,848 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ basic_simple2 (Sequential)      │ (None, 256)            │       393,728 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ classifier (Dense)              │ (None, 6)              │         1,542 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 118,142,125 (450.68 MB)

 Trainable params: 395,270 (1.51 MB)

 Non-trainable params: 117,746,855 (449.17 MB)

In [ ]:
# training the model
eV2L_simple2_history = eV2L_simple2().fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=200,
    callbacks=[
        utils.callback_savemodel('eV2L_simple2.keras'),
        utils.callback_earlystop(30)
    ],
)

Epoch 1/200


I0000 00:00:1737432546.413002  261382 cuda_dnn.cc:529] Loaded cuDNN version 90600


4/4 ━━━━━━━━━━━━━━━━━━━━ 33s 3s/step - accuracy: 0.1687 - loss: 1.8022 - val_accuracy: 0.2344 - val_loss: 1.7676
Epoch 2/200
4/4 ━━━━━━━━━━━━━━━━━━━━ 3s 1s/step - accuracy: 0.1932 - loss: 1.7549 - val_accuracy: 0.3281 - val_loss: 1.7514
Epoch 3/200
4/4 ━━━━━━━━━━━━━━━━━━━━ 3s 909ms/step - accuracy: 0.2285 - loss: 1.7621 - val_accuracy: 0.3594 - val_loss: 1.7391
Epoch 4/200
4/4 ━━━━━━━━━━━━━━━━━━━━ 4s 1s/step - accuracy: 0.2163 - loss: 1.7474 - val_accuracy: 0.3906 - val_loss: 1.7285
Epoch 5/200
4/4 ━━━━━━━━━━━━━━━━━━━━ 3s 992ms/step - accuracy: 0.3534 - loss: 1.7002 - val_accuracy: 0.3594 - val_loss: 1.7156
Epoch 6/200
4/4 ━━━━━━━━━━━━━━━━━━━━ 4s 901ms/step - accuracy: 0.3213 - loss: 1.6965 - val_accuracy: 0.3594 - val_loss: 1.7096
Epoch 7/200
4/4 ━━━━━━━━━━━━━━━━━━━━ 3s 889ms/step - accuracy: 0.3431 - loss: 1.6778 - val_accuracy: 0.4062 - val_loss: 1.6989
Epoch 8/200
4/4 ━━━━━━━━━━━━━━━━━━━━ 3s 868ms/step - accuracy: 0.3492 - loss: 1.6744 - val_accuracy: 0.3594 - val_loss: 1.6869
Epoc

In [ ]:
# visualizing the training process
utils.plot_history(eV2L_simple2_history, loss_ylim=(0, 2))

: 

In [ ]:
# evaluating the model
test_loss, test_acc = keras.models.load_model('eV2L_simple2.keras').evaluate(test_dataset)
print(f"Test accuracy: {test_acc:.2f}%, Test loss: {test_loss:.2f}")

: 

# data_enhanced_0_original_100

In [ ]:
# loading the dataset
train_dataset, val_dataset, test_dataset = utils.load_unified_datasets("data/data_enhanced_0_original_100")

# training the model
eV2L_simple2_history = eV2L_simple2().fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=200,
    callbacks=[
        utils.callback_savemodel('eV2L_simple2_e00o10.keras'),
        utils.callback_earlystop(30)
    ],
)

: 

In [ ]:
# visualizing the training process
utils.plot_history(eV2L_simple2_history, loss_ylim=(0, 2))

: 

In [ ]:
# evaluating the model
test_loss, test_acc = keras.models.load_model('eV2L_simple2_e00o10.keras').evaluate(test_dataset)
print(f"Test accuracy: {test_acc:.2f}%, Test loss: {test_loss:.2f}")

: 

# data_enhanced_30_original_70

In [ ]:
# loading the dataset
train_dataset, val_dataset, test_dataset = utils.load_unified_datasets("data/data_enhanced_30_original_70")

# training the model
eV2L_simple2_history = eV2L_simple2().fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=200,
    callbacks=[
        utils.callback_savemodel('eV2L_simple2_e03o07.keras'),
        utils.callback_earlystop(30)
    ],
)

: 

In [ ]:
# visualizing the training process
utils.plot_history(eV2L_simple2_history, loss_ylim=(0, 2))

: 

In [ ]:
# evaluating the model
test_loss, test_acc = keras.models.load_model('eV2L_simple2_e03o07.keras').evaluate(test_dataset)
print(f"Test accuracy: {test_acc:.2f}%, Test loss: {test_loss:.2f}")

: 

# data_enhanced_50_original_50

In [ ]:
# loading the dataset
train_dataset, val_dataset, test_dataset = utils.load_unified_datasets("data/data_enhanced_50_original_50")

# training the model
eV2L_simple2_history = eV2L_simple2().fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=200,
    callbacks=[
        utils.callback_savemodel('eV2L_simple2_e05o05.keras'),
        utils.callback_earlystop(30)
    ],
)

: 

In [ ]:
# visualizing the training process
utils.plot_history(eV2L_simple2_history, loss_ylim=(0, 2))

: 

In [ ]:
# evaluating the model
test_loss, test_acc = keras.models.load_model('eV2L_simple2_e05o05.keras').evaluate(test_dataset)
print(f"Test accuracy: {test_acc:.2f}%, Test loss: {test_loss:.2f}")

: 

# data_enhanced_70_original_30

In [ ]:
# loading the dataset
train_dataset, val_dataset, test_dataset = utils.load_unified_datasets("data/data_enhanced_70_original_30")

# training the model
eV2L_simple2_history = eV2L_simple2().fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=200,
    callbacks=[
        utils.callback_savemodel('eV2L_simple2_e07o03.keras'),
        utils.callback_earlystop(30)
    ],
)

: 

In [ ]:
# visualizing the training process
utils.plot_history(eV2L_simple2_history, loss_ylim=(0, 2))

: 

In [ ]:
# evaluating the model
test_loss, test_acc = keras.models.load_model('eV2L_simple2_e07o03.keras').evaluate(test_dataset)
print(f"Test accuracy: {test_acc:.2f}%, Test loss: {test_loss:.2f}")

: 

# data_enhanced_100_original_0

In [ ]:
# loading the dataset
train_dataset, val_dataset, test_dataset = utils.load_unified_datasets("data/data_enhanced_100_original_0")

# training the model
eV2L_simple2_history = eV2L_simple2().fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=200,
    callbacks=[
        utils.callback_savemodel('eV2L_simple2_e10o00.keras'),
        utils.callback_earlystop(30)
    ],
)

: 

In [ ]:
# visualizing the training process
utils.plot_history(eV2L_simple2_history, loss_ylim=(0, 2))

: 

In [ ]:
# evaluating the model
test_loss, test_acc = keras.models.load_model('eV2L_simple2_e10o00.keras').evaluate(test_dataset)
print(f"Test accuracy: {test_acc:.2f}%, Test loss: {test_loss:.2f}")

: 